In [1]:
import torch
from torch import nn
from torchvision import models
from torchvision import transforms
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
from torch import optim
from PIL import Image
import os
import torch.nn as nn
from torch.autograd import Variable

In [2]:
data_dir = '/scratch/sa5154/ssl_data_96'

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
def load_data(data_dir, batch_size, split):
    """ Method returning a data loader for labeled data """
    # TODO: add data transformations if needed
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ]
    )
    data = datasets.ImageFolder(f'{data_dir}/supervised/{split}', transform=transform)
    data_loader = DataLoader(
        data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
    return data_loader

In [5]:
def train (input_image,target):
    resnet.train()
    optimizer.zero_grad()
    prob=resnet(input_image)
    loss=criterion(prob,target)
    loss.backward()
    optimizer.step()
    return loss.item()

In [6]:
def validate (input_image,target):
    with torch.no_grad():
        resnet.eval()
        prob=resnet(input_image)
        loss=criterion(prob,target)
        return loss.item()

In [7]:
train_data = load_data(data_dir, 16, 'train')
val_data = load_data(data_dir, 16, 'val')

In [11]:
resnet = models.resnet34(pretrained=False).to(device)

In [12]:
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(FEN.parameters())
optimizer = torch.optim.Adam(resnet.parameters(), lr=1e-1)

In [ ]:
train_loss=[]
validation_loss=[]
epochs = 30
global_loss = 10
for epoch in range(epochs):
    pointer_train = 0
    pointer_test = 0
    pointer_cv = 0

    running_loss = 0
    for i, (X_train, y_train) in enumerate(train_data):
        X_train = X_train.to(device)
        y_train = y_train.to(device)
        loss=train(X_train,y_train)
        running_loss+=loss
        if(i%100 == 0):
            print('Epoch:{} | Iter: {} | Train Loss: {}'.format(epoch + 1, i+1, running_loss/(i+1)))
        
    #Validation
    loss_val=0
    for i, (X_val, y_val) in enumerate(val_data):
        X_val = X_val.to(device)
        y_val = y_val.to(device)
        loss_val+=validate(X_val,y_val)
    
    loss_val=loss_val/len(val_data)
    validation_loss.append(loss_val)
    running_loss = running_loss/len(train_data)
    train_loss.append(running_loss)
    print ('Epoch: {} | Training loss: {} | Validation Loss: {}'.format(epoch+1, running_loss, loss_val))
    if(global_loss > loss_val):
        global_loss = loss_val
        torch.save(FEN.state_dict(), './saved_models/VGG-Adam-LR-3-5-Aug.pth')
        print("Model saved at epoch: {}".format(epoch+1))
                   
    #print ('Epoch: {} | Training loss: {}'.format(epoch+1, running_loss))

Epoch:1 | Iter: 1 | Train Loss: 7.143449306488037
Epoch:1 | Iter: 101 | Train Loss: 7.449985192553831
Epoch:1 | Iter: 201 | Train Loss: 7.225412743601633
Epoch:1 | Iter: 301 | Train Loss: 7.142028607403321
Epoch:1 | Iter: 401 | Train Loss: 7.1015373560555854
Epoch:1 | Iter: 501 | Train Loss: 7.077775888576241
Epoch:1 | Iter: 601 | Train Loss: 7.059805820864964
Epoch:1 | Iter: 701 | Train Loss: 7.046019088864837


In [ ]:
3